In [1]:
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
from catboost import CatBoostRegressor

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

import folium
from geopy.geocoders import Nominatim
import time
import geopandas as gpd
import fiona
def get_coordinates(address):
    geolocator = Nominatim(user_agent = "South Korea")
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return 0
path = '../data/daegu/'
external = path+'external_open/대구 빅데이터 마트 데이터/'
pd.set_option('display.max_columns', None)

In [2]:
#### 횡단보도 동별로 개수 구하기
def get_sigudong_from_centroid(point):
    latlong = list(point.x,point.y)
    
    # NCP 콘솔에서 복사한 클라이언트ID와 클라이언트Secret 값
    client_id = "y9ig6cz3sg"
    client_secret = "1YHOy21rqo1f1gmJkxkUGD8jTDmIZ7Z81IdtLfcY"

    # 좌표 (경도, 위도)
    coords = f"{latlong[0]},{latlong[1]}"
    output = "json"
    orders = 'addr'
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    url = f"{endpoint}?coords={coords}&output={output}&orders={orders}"

    # 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # 요청
    res = requests.get(url, headers=headers)
    j = res.json()
    
    if j['status']['name'] == 'ok':
        si = j['results'][0]['region']['area1']['name']
        gu = j['results'][0]['region']['area2']['name']
        dong = j['results'][0]['region']['area3']['name']
        
    else:
        si = ''
        gu = ''
        dong = ''
    
    return [si,gu,dong]   

In [3]:
traffic = pd.read_csv(path+'동별_시간별_통행량.csv',encoding='utf-8')
traffic

,Unnamed: 0.1,Unnamed: 0,구,동,시간,통행량
0,0,0,남구,대명동,0,492
1,1,1,남구,대명동,1,402
2,2,2,남구,대명동,2,0
3,3,3,남구,대명동,3,204
4,4,4,남구,대명동,4,76
...,...,...,...,...,...,...
4075,4075,4075,중구,화전동,19,145747
4076,4076,4076,중구,화전동,20,120750
4077,4077,4077,중구,화전동,21,100877
4078,4078,4078,중구,화전동,22,86046


In [34]:
traffic.isnull().sum()

Unnamed: 0    0
구             0
동             0
시             0
통행량           0
dtype: int64

In [35]:
garomang_path = path+'대구광역시_가로망주정차현황_주차운영및이용실태정보(SHP)_20211210/'
garomang = gpd.read_file(garomang_path+'대구_가로망주정차.shp',encoding='cp949')
garomang

,GIS_ID,MOCT_ID,G_RD_NAM,G_RD_BT,G_RD_LEN,G_GUCODE,G_GUNAM,RVDATE,S_PRK_TYPE,S_PRK_OP_Y,S_PRK_WK_S,S_PRK_WK_E,S_PRK_SA_S,S_PRK_SA_E,S_PRK_SN_S,S_PRK_SN_E,S_PRK_DESC,C_CV_YN,C_CV_WK_ST,C_CV_WK_ED,C_CV_SA_ST,C_CV_SA_ED,C_CV_SN_ST,C_CV_SN_ED,C_CV_DESC,C_SCH_ZONE,C_SCH_DESC,U_PRK_DY,U_PRK_D_L,U_PRK_D_IL,U_PRK_D_IN,U_PRK_D_OT,U_PRK_D_SD,U_PRK_D_ST,U_PRK_D_BT,U_PRK_NT,U_PRK_N_L,U_PRK_N_IL,U_PRK_N_IN,U_PRK_N_OT,U_PRK_N_SD,U_PRK_N_ST,U_PRK_N_BT,U_PRK_DATE,geometry
0,1,None,방촌로,12.0,4.208,02,동구,2020-08,② 황색실선_단선,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (168925.601 365574.266, 168926.484 ..."
1,2,None,동촌로,35.0,25.136,02,동구,2020-08,① 황색실선_복선,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169521.136 365079.639, 169541.425 ..."
2,3,None,입석로,12.0,240.835,02,동구,2020-08,⑤ 주차면,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (168974.853 365622.076, 168929.316 ..."
3,4,None,동촌로,35.0,3.306,02,동구,2020-08,⑦ 횡단보도,0,None,None,None,None,None,None,None,1,07:00,20:00,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169552.936 365057.295, 169555.120 ..."
4,5,None,화랑로80길,14.0,5.594,02,동구,2020-08,⑦ 횡단보도,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,1,"나현유치원, 다애유치원",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169737.701 364264.717, 169733.591 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178852,179366,None,동촌로,35.0,2.439,02,동구,2020-08,⑦ 횡단보도,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169467.814 365108.961, 169467.419 ..."
178853,179367,None,동촌로,35.0,23.458,02,동구,2020-08,⑦ 횡단보도,0,None,None,None,None,None,None,None,1,07:00,20:00,07:00,20:00,07:00,20:00,"토,공휴일 포함",0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (168567.901 365574.438, 168570.637 ..."
178854,179368,None,동촌로,35.0,13.295,02,동구,2020-08,① 황색실선_복선,0,None,None,None,None,None,None,None,0,None,None,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169474.047 365105.468, 169474.641 ..."
178855,179369,None,동촌로42길,6.0,2.275,02,동구,2020-08,⑦ 횡단보도,0,None,None,None,None,None,None,None,1,07:00,20:00,None,None,None,None,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (169515.970 365010.689, 169513.782 ..."


In [49]:
daegu_df

,구,동,위도,경도
0,남구,대명동,35.843894,128.576926
1,남구,봉덕동,35.839234,128.600070
2,남구,성당동,35.843384,128.561788
3,남구,이천동,35.849420,128.603435
4,달서구,갈산동,35.840990,128.505850
...,...,...,...,...
197,중구,태평로2가,35.875649,128.591902
198,중구,태평로3가,35.876462,128.587159
199,중구,포정동,35.871923,128.594115
200,중구,향촌동,35.873418,128.593691


In [4]:
daegu_df = pd.read_csv(path+'구동별위도경도.csv',encoding='cp949')

def make_point(x):
    lat = x['위도']
    long = x['경도']
    point=Point((long,lat))
    return point
    
daegu_df['geometry'] = daegu_df.apply(make_point,axis=1)

geo_daegu = gpd.GeoDataFrame(daegu_df, geometry='geometry')

NameError: name 'Point' is not defined

In [70]:
daegu_df = pd.read_csv(path+'구동별위도경도.csv',encoding='cp949')
daegu_df.head()

,Unnamed: 0.1,Unnamed: 0,구,동,위도,경도,geometry
0,0,0,남구,대명동,35.843894,128.576926,POINT (128.5769255645161 35.84389397725807)
1,1,1,남구,봉덕동,35.839234,128.600070,POINT (128.6000701166667 35.83923358833334)
2,2,2,남구,성당동,35.843384,128.561788,POINT (128.5617884 35.84338382)
3,3,3,남구,이천동,35.849420,128.603435,POINT (128.603435375 35.84941956625)
4,4,4,달서구,갈산동,35.840990,128.505850,POINT (128.5058504 35.84098981)


In [72]:
type(daegu_df['geometry'][0])

str

In [71]:
daegu_gdf = gpd.GeoDataFrame(daegu_df,geometry='geometry')
daegu_gdf['geometry'] = daegu_gdf['geometry'].apply(lambda x:x.buffer(0.1)) 
daegu_gdf



TypeError: Input must be valid geometry objects: POINT (128.5769255645161 35.84389397725807)

In [44]:
from shapely.geometry import Polygon, MultiPolygon, Point

###### reverse_geocoding을 구동별위도경도가 Polygon에 있는지 여부로 확인하기
def reverse_geocoding(poly, daegu_df):
    isDone = False
    if isinstance(poly, Polygon):
        for row_index in range(len(daegu_df)):
            point = daegu_df.iloc[row_index,'geometry']
            
        pass
    else: # MultiPolygon
        for p in poly:
            pass
        
    
    
    
    
    

In [61]:
# 대구광역시 행정동
daegu_geo1 = gpd.read_file(external+'0. Base/대구광역시_100.gpkg',encoding='cp949')
daegu_geo2 = gpd.read_file(external+'0. Base/대구광역시_300.gpkg',encoding='cp949')
daegu_geo3 = gpd.read_file(external+'0. Base/대구광역시_500.gpkg',encoding='cp949')

daegu_geo = pd.concat([daegu_geo1,daegu_geo2,daegu_geo3],axis=0)
daegu_geo = daegu_geo.to_crs(epsg='4326') # crs 정보 위경도 좌표계로 변경
daegu_geo.set_index('id',inplace=True)
daegu_geo.sort_index(inplace=True)
daegu_geo.reset_index(inplace=True)
daegu_geo.head(2)

,id,left,top,right,bottom,geometry
0,69,1.077033e+06,1.746587e+06,1.077533e+06,1.746087e+06,"MULTIPOLYGON (((128.35675 35.70806, 128.35710 ..."
1,70,1.077033e+06,1.746087e+06,1.077533e+06,1.745587e+06,"MULTIPOLYGON (((128.35180 35.70402, 128.35230 ..."
